## Example how CmdInterface deals with input and output logging

In [1]:
from cmdint import CmdInterface
import os

First we remove some files that might have been created in a previous run of this script and we want a clean start.

In [2]:
if os.path.isfile('io_file_checking.txt'):
    os.remove('io_file_checking.txt')
if os.path.isfile('io_file_checking_copy.txt'):
    os.remove('io_file_checking_copy.txt')

Set output logfile and tell CmdInterface to delete a potentially present old logfile

In [3]:
CmdInterface.set_static_logfile('io_file_checking.json', delete_existing=True)

We want to log if a run is not necessary. By default this is disabled since we don't want to swamp our logfile if we call the script multiple times.

In [4]:
CmdInterface.set_immediate_return_on_run_not_necessary(do_return=False)

For this example we also dont't want to exit python if a run fails. The default is True.

In [5]:
CmdInterface.set_exit_on_error(do_exit=False)

### Step 1:Create a new text file using *echo*

In [6]:
step1 = CmdInterface('echo')
step1.add_arg(arg='unus ignis quis vir multum ab audere')

Warning, repo /home/neher/cmdint/cmdint is dirty!


By setting *check_output* to True, the argument is treated as an expected output file (logged under `['command']['output']['expected']`). If the file is found **BEFORE** execution, the execution is skipped, since a run is regarded as unnecessary when all
output files are already present. The run fails if this file is not found **AFTER** execution (logged under: `['command']['output']['missing']`). If it is found, the run is successful and the file hash is logged to enable unique identification (logged under `['command']['output']['found']`).

In [7]:
step1.add_arg(key='>', arg='io_file_checking.txt', check_output=True)
step1.run()

2019-02-05 08:47:47 >> echo START
2019-02-05 08:47:47 >> echo END


1

### Step 2: copy the previously created file using *cp*

In [8]:
step2 = CmdInterface('cp')

Warning, repo /home/neher/cmdint/cmdint is dirty!


Similar to *check_output*, the *check_input* flag declares an argument as expected input file (logged under `['command']['input']['expected']`). The run fails if this file is not found **BEFORE** execution (logged under: `['command']['input']['missing']`). If it is found, the run is started and the file hash is logged to enable unique identification (logged under `['command']['input']['found']`).

In [9]:
step2.add_arg(arg='io_file_checking.txt', check_input=True)
step2.add_arg(arg='io_file_checking_copy.txt', check_output=True)
step2.run()

2019-02-05 08:53:18 >> cp START
2019-02-05 08:53:18 >> cp END


1

Let's run step2 again to get a log of a skipped run since the output file is already there. The log of this command is simply appended to the json file because we did not specify a new file name.

In [10]:
step2.run()

2019-02-05 08:54:33 >> cp START
Skipping execution. All output files already present.
2019-02-05 08:54:33 >> cp END


2

### Step 3: let's create a run that will fail due to a missing input file

In [11]:
step3 = CmdInterface('cp')
step3.add_arg(arg='non_existing_file.txt', check_input=True)
step3.add_arg(arg='non_existing_file_copy.txt', check_output=True)
step3.run()

Warning, repo /home/neher/cmdint/cmdint is dirty!
2019-02-05 08:55:13 >> cp START
Skipping execution. Input files missing: ['non_existing_file.txt']
2019-02-05 08:55:13 >> cp END


-2

And finally, anonymize our logfile again as explained in `simply.ipynb`.

In [12]:
CmdInterface.anonymize_log(out_log_name='io_file_checking.json')

Anonymizing io_file_checking.json
